In [1]:
import os 
from dotenv import load_dotenv

#load variables from the .env file
load_dotenv()

# Get the key from the environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key set successfully.")
else: 
    print("GOOGLE_API_KEY not found in environment variables.")

Gemini API key set successfully.


In [2]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

🧹 Cleaned up logger.log
✅ Logging configured


# Section 2: Hands-On Debugging with ADK Web UI
## Create a "Research Paper Finder" Agent
Our goal: Build a research paper finder agent that helps users find academic papers on any topic.

But first, let's intentionally create an incorrect version of the agent to practice debugging! We'll start by creating a new agent folder using the adk create CLI command.

In [3]:
# !adk create research_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [4]:
%%writefile research_agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# ---- Intentionally pass incorrect datatype - `str` instead of `List[str]` ----
def count_papers(papers: str):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google Search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)


# Root agent
root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.
    """,
    tools=[AgentTool(agent=google_search_agent), count_papers]
)

Overwriting research_agent/agent.py


In [5]:
# Temporary workaround if you're local
def get_adk_proxy_url():
    return "http://localhost:8090" # Or your specific endpoint
url_prefix = get_adk_proxy_url()

In [6]:
import subprocess
import os

def kill_port_windows(port):
    try:
        # 1. Find the line in netstat that has our port
        # 'netstat -ano' shows protocol, address, state, and PID
        result = subprocess.check_output(f'netstat -ano | findstr :{port}', shell=True).decode()
        
        lines = result.strip().split('\n')
        for line in lines:
            if "LISTENING" in line:
                # 2. Extract the PID (it's the last number in the row)
                pid = line.strip().split()[-1]
                print(f"Found process {pid} listening on port {port}. Killing it...")
                
                # 3. Kill the process forcefully (/F)
                os.system(f"taskkill /F /PID {pid}")
                print(f"✅ Port {port} is now free.")
                return
    except subprocess.CalledProcessError:
        print(f"ℹ️ Port {port} appears to be already free.")
    except Exception as e:
        print(f"❌ Error: {e}")

kill_port_windows(8090)

ℹ️ Port 8090 appears to be already free.


In [7]:

#!adk web --port 8090 --log_level DEBUG --url_prefix {url_prefix}

In [8]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


def count_papers(papers: List[str]):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="Use the google_search tool to find information on the given topic. Return the raw search results.",
    tools=[google_search],
)

# Root agent
research_agent_with_plugin = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 
   
   You must follow these steps:
   1) Find research papers on the user provided topic using the 'google_search_agent'. 
   2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
   3) Return both the list of research papers and the total number of papers.
   """,
    tools=[AgentTool(agent=google_search_agent), count_papers],
)

print("✅ Agent created")


✅ Agent created


In [9]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=research_agent_with_plugin,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)

print("✅ Runner configured")

✅ Runner configured


In [10]:
print("🚀 Running agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await runner.run_debug("Find recent papers on quantum computing")

🚀 Running agent with LoggingPlugin...
📊 Watch the comprehensive logging output below:


 ### Created new session: debug_session_id

User > Find recent papers on quantum computing
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f4650590-8573-4aac-ab27-9d076c61e1cf
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: research_paper_finder_agent
[logging_plugin]    User Content: text: 'Find recent papers on quantum computing'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f4650590-8573-4aac-ab27-9d076c61e1cf
[logging_plugin]    Starting Agent: research_paper_finder_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: research_paper_finder_agent
[logging_plugin]    Invocation ID: e-f4650590-8573-4aac-ab27-9d076c61e1cf
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[l

c:\Python313\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: google_search_agent
[logging_plugin]    Content: text: 'Recent papers on quantum computing highlight significant progress in 2023 and 2024, with a focus on increasing qubit stability and developing better error correction techniques. While quantum computin...'
[logging_plugin]    Token Usage - Input: 58, Output: 407
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 6ef7c6ba-7f39-435a-aa25-d2dcbcb38639
[logging_plugin]    Author: google_search_agent
[logging_plugin]    Content: text: 'Recent papers on quantum computing highlight significant progress in 2023 and 2024, with a focus on increasing qubit stability and developing better error correction techniques. While quantum computin...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: google_search_agent
[logging_plugin]    Invocation ID: e-a440609f-9376-4ab2-adc4-4b0438be9b4e
[logging_plugin] ✅ INVOCATION CO